In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *
import sys

# Lambda_idx = int(sys.argv[1])
# n_external = int(sys.argv[2])
# ell_idx = int(sys.argv[3])


Lambda_idx = 18
n_external = 500
ell_idx = 0

Lambda = Lambdas[Lambda_idx]

# CCAT-prime
zmin = 3.5
zmax = 8.1

oup_fname = '/scratch/users/delon/LIMxCMBL/I_auto/ILo_ILo_zmin_%.1f_zmax_%.1f_Lambda_idx_%d_n_ext_%d_l_%d_quad.npy'%(zmin, zmax, 
                                                                                                                     Lambda_idx, 
                                                                                                                     n_external,
                                                                                                                    ell_idx)
print(oup_fname)

/scratch/users/delon/LIMxCMBL/I_auto/ILo_ILo_zmin_3.5_zmax_8.1_Lambda_idx_18_n_ext_500_l_0_quad.npy


In [2]:
Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))



In [3]:
from scipy.interpolate import interp1d
from scipy.integrate import quad, quad_vec, trapezoid

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
f_inner_integral = interp1d(x = chibs, y = inner_dkparp_integral[ell_idx], axis = 0)
external_chis = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), n_external)
print('external chi spacing', np.mean(np.diff(external_chis)))

external chi spacing 4.427931588483547


In [4]:
external_chis

array([6963.73474175, 6968.16267334, 6972.59060493, 6977.01853652,
       6981.44646811, 6985.8743997 , 6990.30233128, 6994.73026287,
       6999.15819446, 7003.58612605, 7008.01405764, 7012.44198923,
       7016.86992082, 7021.2978524 , 7025.72578399, 7030.15371558,
       7034.58164717, 7039.00957876, 7043.43751035, 7047.86544193,
       7052.29337352, 7056.72130511, 7061.1492367 , 7065.57716829,
       7070.00509988, 7074.43303147, 7078.86096305, 7083.28889464,
       7087.71682623, 7092.14475782, 7096.57268941, 7101.000621  ,
       7105.42855258, 7109.85648417, 7114.28441576, 7118.71234735,
       7123.14027894, 7127.56821053, 7131.99614212, 7136.4240737 ,
       7140.85200529, 7145.27993688, 7149.70786847, 7154.13580006,
       7158.56373165, 7162.99166324, 7167.41959482, 7171.84752641,
       7176.275458  , 7180.70338959, 7185.13132118, 7189.55925277,
       7193.98718435, 7198.41511594, 7202.84304753, 7207.27097912,
       7211.69891071, 7216.1268423 , 7220.55477389, 7224.98270

In [5]:
ILoILo = np.zeros((len(ells), (n_external), (n_external)), dtype=np.float64)

from tqdm import trange

f_KLIMLo1 = get_f_KILo(external_chi = external_chis.reshape(n_external,1,1), Lambda=Lambda)
f_KLIMLo1_windowed = apply_window(f_K = f_KLIMLo1,
                                 chimin = chimin,
                                 chimax = chimax)

f_KLIMLo2 = get_f_KILo(external_chi = external_chis.reshape(1,n_external, 1), Lambda=Lambda)
f_KLIMLo2_windowed = apply_window(f_K = f_KLIMLo2,
                                 chimin = chimin,
                                 chimax = chimax)


def integrand(_chib):
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)

    _interm  = f_KLIMLo1_windowed(plus) * f_KLIMLo2_windowed(mins)
    _interm += f_KLIMLo1_windowed(mins) * f_KLIMLo2_windowed(plus) 

    _factor = (2 / _chib)
    _factor = _factor * deltas
    _factor = np.einsum('d, d->d', _factor, f_inner_integral(_chib))

    _interm  = np.einsum('xyd,d->xyd', _interm, _factor)

    return trapezoid(x = np.log(deltas), y = _interm, axis=-1)

In [6]:
res, _ = quad_vec(integrand, 10, chimax_sample, epsrel = 1e-3, epsabs =0.0, 
                 workers=32)

In [7]:
np.save(oup_fname, res)
print('outputted')

outputted
